In [ ]:
import os
import shutil

original_db = "database/hotel_reservation_sample.db"
working_db = "database/hotel_reservation.db"
shutil.copy(original_db, working_db)

os.environ["DB"] = working_db

# Userstory 1
Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit
ich dasjenige auswählen kann, welches meinen Wünschen
entspricht. Wünsche sind:

# Userstory 1.1
Ich möchte alle Hotels in einer Stadt durchsuchen,
damit ich das Hotel nach meinem bevorzugten Standort
(Stadt) auswählen kann.


In [ ]:
import model
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager

hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    hotels = hotel_manager.search_hotels_by_city(city)
    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"{hotel.name} mit {hotel.stars} Sternen an der {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}")
        valid = True
    else:
        print("Keine Hotels gefunden")


# Userstory 1.2
Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.


In [ ]:
import model
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    stars = int(input("Geben Sie die min. Anzahl Sterne an" ))
    hotels = hotel_manager.search_hotels_by_city_and_stars(city, stars)
    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"{hotel.name} mit {hotel.stars} Sternen an der {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}")
        valid = True
    else:
        print("Keine Hotels gefunden")


# Userstory 1.3
Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung).


In [ ]:
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    try:
        guest_count = int(input("Geben Sie die min. Anzahl der Gäste an: "))
    except ValueError:
        print("Ungültige Eingabe, geben Sie eine Zahl an")

    hotels = hotel_manager.search_hotels_by_city_and_room_capacity(city, guest_count)

    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"\n{hotel.name}, Sterne: {hotel.stars}"
                  f"\nAddresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
                  f"\nZimmer: {hotel.room_number}, {hotel.description}, Kapazität: {hotel.max_guests}"
                  f"\nPreis pro Nacht: {hotel.price_per_night}")

        valid = True
    else:
        print("Keine Hotels gefunden")


# Userstory 1.4
Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.


In [ ]:
from datetime import datetime
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()
valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    check_in_date = input("Geben Sie ihr gewünschtes Anreisedatum an (YYYY-MM-DD): ")
    check_out_date = input("Geben Sie ihr gewünschtes Abreisedatum an (YYYY-MM-DD): ")

    try:
        check_in_date = datetime.strptime(check_in_date, "%Y-%m-%d").date()
        check_out_date = datetime.strptime(check_out_date, "%Y-%m-%d").date()
    except ValueError:
        print("Ungültiges Datumsformat. Bitte unbedingt YYYY-MM-DD verwenden.\n")
        continue

    if check_in_date >= check_out_date:
        print("Fehler: Checkout muss nach dem Checkin liegen. Bitte erneut eingeben")
        continue

    hotels = hotel_manager.search_hotels_by_city_and_availability(city, check_in_date, check_out_date)

    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"\n{hotel.name}, Sterne: {hotel.stars}"
                  f"\nAddresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
                  f"\nZimmer: {hotel.room_number}, {hotel.description}, Kapazität: {hotel.max_guests}"
                  f"\nPreis pro Nacht: {hotel.price_per_night}"
                  f"\nAnreisedatum von bis: {check_in_date} - {check_out_date}"
            )
        valid = True
    else:
        print("Leider keine Verfügbarkeit in diesem Zeitraum. Versuchen Sie bitte einen anderen Zeitraum.")



# Userstory 1.5
Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne

In [ ]:
from datetime import datetime
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()
valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    try:
        check_in_str = input("Geben Sie ihr gewünschtes Anreisedatum an (YYYY-MM-DD): ")
        check_out_str = input("Geben Sie ihr gewünschtes Abreisedatum an (YYYY-MM-DD): ")
        check_in_date = datetime.strptime(check_in_str, "%Y-%m-%d").date()
        check_out_date = datetime.strptime(check_out_str, "%Y-%m-%d").date()
    except ValueError:
        print("Ungültiges Datumsformat. Bitte unbedingt YYYY-MM-DD verwenden.\n")
        continue

    if check_in_date >= check_out_date:
        print("Fehler: Checkout muss nach dem Checkin liegen. Bitte erneut eingeben")
        continue

    try:
        stars = int(input("Geben Sie die min. Anzahl Sterne an" ))
    except ValueError:
        print("Ungültige Eingabe, geben Sie eine Zahl an")
        continue

    try:
        guest_count = int(input("Geben Sie die min. Anzahl der Gäste an: "))
    except ValueError:
        print("Ungültige Eingabe, geben Sie eine Zahl an")
        continue


    hotels = hotel_manager.search_hotels_by_city_availability_stars_capacity(
        city,
        check_in_date,
        check_out_date,
        stars,
        guest_count
    )

    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"\n{hotel.name}, Sterne: {hotel.stars}"
                  f"\nAddresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
                  f"\nZimmer: {hotel.room_number}, {hotel.description}, Kapazität: {hotel.max_guests}"
                  f"\nPreis pro Nacht: {hotel.price_per_night}"
                  f"\nAnreisedatum von bis: {check_in_date} - {check_out_date}"
            )
        valid = True
    else:
        print("Leider keine Verfügbarkeit in diesem Zeitraum. Versuchen Sie bitte einen anderen Zeitraum.")



# Userstory 1.6
Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [ ]:
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

hotels = hotel_manager.get_all_hotel_info()
if not hotels:
    print("Keine Hotels gefunden.")
else:
    for hotel in hotels:
        print(f"\nHotel: {hotel.name}, Sterne: {hotel.stars}\n"
              f"Adresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
            )


# Userstory 2
Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.
# Userstory 2.1
Ich möchte die folgenden Informationen pro Zimmer
sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung,
Ausstattung, Preis pro Nacht und Gesamtpreis.

In [ ]:

from business_logic.room_manager import RoomManager

room_manager = RoomManager()

hotel_name_input = input("Geben Sie den exakten Hotelnamen ein: ").strip()
try:
    rooms = room_manager.get_room_info_by_hotel(hotel_name_input)
except Exception as e:
    print(f"Fehler bei der Abfrage: {e}")

if not rooms:
    print(f"Für Hotel „{hotel_name_input}“ wurden keine Zimmer gefunden.")
else:
    print(f"\nZimmer für „{hotel_name_input}“:\n")
    for r in rooms:
        print(
            f"  Zimmer-ID      : {r.room_id}\n"
            f"  Zimmernummer   : {r.room_number}\n"
            f"  Zimmertyp      : {r.room_type.description}\n"
            f"  Max. Gäste     : {r.room_type.max_guests}\n"
            f"  Preis/Nacht    : {r.price_per_night}\n")

# Userstory 2.2
Ich möchte nur die verfügbaren Zimmer sehen, sofern
ich meinen Aufenthalt (von – bis) spezifiziert habe.


In [ ]:
from datetime import datetime
from business_logic.room_manager import RoomManager  # oder: HotelManager

room_manager = RoomManager()


valid = False
while not valid:
    try:
        check_in_str = input("Geben Sie ihr gewünschtes Anreisedatum an (YYYY-MM-DD): ")
        check_out_str = input("Geben Sie ihr gewünschtes Abreisedatum an (YYYY-MM-DD): ")
        check_in_date = datetime.strptime(check_in_str, "%Y-%m-%d").date()
        check_out_date = datetime.strptime(check_out_str, "%Y-%m-%d").date()
    except ValueError:
        print("Ungültiges Datumsformat. Bitte unbedingt YYYY-MM-DD verwenden.\n")
        continue

    if check_out_date <= check_in_date:
        print("Fehler: Check-out muss nach Check-in liegen. Bitte erneut eingeben.\n")
        continue

    rooms = room_manager.get_available_rooms_by_date(check_in_date, check_out_date)
    valid = True

if not rooms:
    print(f"Im Zeitraum {check_in_date}–{check_out_date} wurden keine freien Zimmer gefunden.")
else:
    print(f"\nVerfügbare Zimmer für ({check_in_date} bis {check_out_date}):\n")
    for r in rooms:
        print(
            f"- Zimmer-ID       : {r.room_id}\n"
            f"  Zimmernummer    : {r.room_number}\n"
            f"  Zimmertyp       : {r.room_type.description}\n"
            f"  Max. Gäste      : {r.room_type.max_guests}\n"
            f"  Preis pro Nacht : {r.price_per_night}\n"
        )

# Userstory 3
Als Admin des Buchungssystems möchte ich die Möglichkeit haben,
Hotelinformationen zu pflegen, um aktuelle Informationen im
System zu haben.
# Userstory 3.1
Ich möchte neue Hotels zum System hinzufügen

## Input Daten vom Admin

* Daten wie Strasse, Hotelname, Anzahl Sterne und Ort(Addresse)




In [ ]:
# Userstory 3.1
from datetime import date
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager
from model.hotel import Hotel
from model.address import Address

address_manager = AddressManager()
createded_address = address_manager.create_address("Teststrasse", "Zürich", 8001)
print(f"Neue Addresse: {createded_address.street}, {createded_address.city}, {createded_address.zip_code}")


hotel_manager = HotelManager()
h = Hotel(6, "Olympia", 3)
hotel = hotel_manager.create_hotel("Olympia", 3, createded_address)
print(f"Neues Hotel: {h.name}, Sterne: {h.stars}")

# Userstory 3.2
Ich möchte Hotels aus dem System entfernen


In [ ]:
# TODO wenn Hotel gelöscht wird, muss auch Adresse aus dem System raus -> muss noch implementiert werden
import time
from business_logic.hotel_manager import HotelManager

hotels = hotel_manager.get_all_hotel_info()

if not hotels:
    print("Keine Hotels in der Datenbank gefunden")
else:
    for hotel in hotels:
        print(f"{hotel.hotel_id}: {hotel.name}, {hotel.stars}")
    time.sleep(0.5)

valid = False
while not valid:
    try:
        user_choice = int(input("Geben Sie die ID ein die Sie löschen möchten"))
        if user_choice <= 0 or user_choice > len(hotels):
            raise ValueError()
        else:
            valid = True
    except ValueError:
        print(f"Ungültige Eingabe bitte geben sie eine gültige ID ein")

try:
    hotel_manager.delete_hotel(user_choice)
    print(f"Hotel mit ID: {user_choice} wurde erfolgreich gelöscht.\n")
except Exception as e:
    print(f"Fehler beim Löschen: {e}")

hotels = hotel_manager.get_all_hotel_info()
if not hotels:
    print("Keine Hotels in der Datenbank gefunden")
else:
    print("Aktualisierte Liste: ")
    for hotel in hotels:
        print(f"{hotel.hotel_id}: {hotel.name}, {hotel.stars}")



# Userstory 3.3
Ich möchte die Informationen bestimmter Hotels
aktualisieren, z. B. den Namen, die Sterne usw.


In [ ]:
import time
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

hotels = hotel_manager.get_all_hotel_info()
if not hotels:
    print("Keine Hotels in der Datenbank vorhanden.")
else:
    for idx, hotel in enumerate(hotels, start=1):
        print(f"{idx} - {hotel.name}, {hotel.stars}")
    time.sleep(0.5)

valid = False
while not valid:
    try:
        index = int(input("Geben Sie die Nummer ein, die Sie aktualisieren möchten: ")) - 1
        if index < 0 or index >= len(hotels):
            raise ValueError()
        else:
            valid = True
    except ValueError:
        print(f"Ungültige Eingabe. Bitte geben Sie eine ganze Zahl zwischen 1 und {len(hotels)} ein.\n")

valid = False
while not valid:
    try:
        name = input("Geben Sie den neuen Hotelnamen ein: ").strip()
        if not name:
            raise ValueError()
        else:
            valid = True
    except ValueError:
        print("Der Name muss ein String sein.")

valid = False
while not valid:
    try:
        stars = int(input("Geben Sie die neue Sternezahl (1–5) ein: "))
        if not 1 <= stars <= 5:
            raise ValueError()
        else:
            valid = True
    except ValueError:
        print("Bitte eine Zahl zwischen 1 und 5 eingeben.\n")

hotel = hotels[index]
hotel.name = name
hotel.stars = stars

try:
    hotel_manager.update_hotel(hotel)
except Exception as e:
    print(f"Fehler beim Aktualisieren: {e}")

print("\nHotel wurde aktualisiert:")
print(f"- ID     : {hotel.hotel_id}")
print(f"- Name   : {hotel.name}")
print(f"- Sterne : {hotel.stars}\n")

for hotel in hotels:
    print(f"{hotel.hotel_id} - {hotel.name}, {hotel.stars}")

# Userstory 4
Als Gast möchte ich ein Zimmer in einem bestimmten Hotel
buchen, um meinen Urlaub zu planen.


In [ ]:
from business_logic.booking_manager import BookingManager
from model.guest import Guest
from model.room import Room
from model.room_type import RoomType
from datetime import date


guest = Guest(1, "John", "Doe", "mail")
room = Room(1, 202, 80.90)
room_type = RoomType(1, "Standard", 1)
room.room_type = room_type

booking_manager = BookingManager()
b = booking_manager.create_booking(guest, room,1, date(2025, 1, 1), date(2025, 1, 3))
print(f" Checkin Date: {b.check_in_date}, Amount: {b.total_amount}")

# Userstory 5
Als Gast möchte ich nach meinem Aufenthalt eine Rechnung
erhalten, damit ich einen Zahlungsnachweis habe.
Hint: Fügt einen Eintrag in der «Invoice» Tabelle hinzu.


In [ ]:
# Import model
from business_logic.invoice_manager import InvoiceManager
from model.invoice import Invoice
from data_access.invoice_dal import InvoiceDAL
from datetime import date
# Input Data from User
# Booking.bookingid: save in variable Bookingid
valid = False
while not valid:
    input_user = input("Geben Sie bitte Ihren Booking ID ein: ")
    if len(input_user) > 0:
        valid = True
    else:
        print("Bitte geben Sie einen Booking ID ein.")
print(input_user)

booking_id = int(input_user)
issue_date = "2025-01-01"
total_amount = "100.00"

invoice_manager = InvoiceManager()

Rechnung = invoice_manager.create_invoice(booking_id, "2025-1-1", 100.00)
if Rechnung is not None:
    print(
        f"Rechnung erstellt: {Rechnung.invoiceid}, BookingNr: {booking_id}, Ausstellungsdatum: {Rechnung.issue_date}, Gesamtbetrag: {Rechnung.total_amount}")
else:
    print("Es konnte keine Rechnung erstellt werden.")

# Userstory 6
Als Gast möchte ich meine Buchung stornieren, damit ich nicht
belastet werde, wenn ich das Zimmer nicht mehr benötige.
Hint: Sorgt für die entsprechende Invoice.

In [ ]:
# Fake Booking erstellen
from business_logic.booking_manager import BookingManager
from model.booking import Booking

valid = False
while not valid:
    input_user = input("Geben Sie bitte Ihren Booking ID ein: ")
    if len(input_user) > 0:
        valid = True
    else:
        print("Bitte geben Sie einen Booking ID ein.")
print(input_user)

booking_id = int(input_user)
booking_manager = BookingManager()

Stornierung = booking_manager.cancel_booking(booking_id)
print("Stornierung erfolgreich")

# Userstory 7
Als Gast möchte ich eine dynamische Preisgestaltung auf der
Grundlage der Nachfrage sehen, damit ich ein Zimmer zum besten
Preis buchen kann.
Hint: Wendet in der Hochsaison höhere und in der Nebensaison
niedrigere Tarife an.


In [ ]:
from datetime import datetime
from data_access.booking_dal import BookingDAL
from model.booking import Booking
from model.guest import Guest


# Ask the User to input a date
user_input = input("When is your check-in date?")
try:
    user_input = datetime.strptime(user_input, "%Y-%m-%d").date()
    print(f"The entered Date is {user_input}")

except ValueError:
    print("Invalid date format. Please enter a valid date in the format YYYY-MM-DD.")

calculate_dynamic_price = BookingDAL().calculate_dynamic_price
price = calculate_dynamic_price(100.00, user_input)

print(f"The price is {price}")

# Userstory 8
Als Admin des Buchungssystems möchte ich alle Buchungen aller
Hotels sehen können, um eine Übersicht zu erhalten.

In [ ]:
from datetime import date
from business_logic.booking_manager import BookingManager
from model.guest import Guest
from model.hotel import Hotel

booking_manager = BookingManager()
bookings = booking_manager.get_all_bookings()

for b in bookings:
    print(
        f"Booking-ID: {b.bookingid} "
        f"Gast: {b.guest.first_name} {b.guest.last_name} "
        f"Hotel: {b.room.hotel.name} "
        f"Zimmernummer: {b.room.room_number} "
        f"Check-In: {b.check_in_date} "
        f"Check-Out: {b.check_out_date} "
    )

# Userstory 9
Als Admin möchte ich eine Liste der Zimmer mit ihrer
Ausstattung sehen, damit ich sie besser bewerben kann.


In [ ]:
from business_logic.room_manager import RoomManager

room_manager = RoomManager()
rooms = room_manager.get_all_rooms_with_equipment()

if rooms:
    print("Zimmer mit Ausstattung:")
    for room in rooms:
        equipment = ", ".join([e.name for e in room.equipment]) if room.equipment else "Keine Ausstattung"
        print(f"Zimmernummer: {room.number}, Hotel: {room.hotel.name}, Typ: {room.room_type.name}, Ausstattung: {equipment}")
else:
    print("Keine Zimmer gefunden.")

# User Stories mit Datenvisualisierung



 Als Admin möchte ich die Belegungsraten für jeden Zimmertyp in
meinem Hotel sehen, damit ich weiss, welche Zimmer am
beliebtesten sind und ich meine Buchungsstrategien optimieren
kann.
Hint: Wählt ein geeignetes Diagramm, um die Auslastung nach
Zimmertyp darzustellen (z. B. wie oft jeder Zimmertyp gebucht
wird).
